In [1]:
from treasury_ml_utils.snowflake_utils import SnowflakeConnector

snowflake_client = SnowflakeConnector("joshua.choy@transferwise.com")

 pip install snowflake-connector-python[secure-local-storage]


Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://transferwise.okta-emea.com/app/snowflake/exk4istpb5gZUyV8u0i7/sso/saml?SAMLRequest=nZJbc9owEIX%2Fikd9tiU7pIAGyHCZTJlJWgomE%2FIm7AVUZMnRyjH011fm0kkfkoe%2BeeSz%2B53ds727Q6GCN7Aoje6TOGIkAJ2ZXOptnyzT%2B7BDAnRC50IZDX1yBCR3gx6KQpV8WLmdnsNrBegC30gjb370SWU1NwIlci0KQO4yvhg%2BPvAkYlwggnUeRy4lOUrP2jlXckrruo7qm8jYLU0YY5R1qVc1ki%2FkHaL8nFFa40xm1LXk4Gf6ABFT1moQXuEJs0vhSOrzCj6jrM8i5N%2FSdBbOfixSEgyv042NxqoAuwD7JjNYzh%2FOBtA7mKdJu5V0IqjCDLSzQoVxhNrUGyX2kJmirJxvHPkvuoGcKrOVfl3TSZ%2BUe5lvjjgab59%2FiS4rl3r3%2Bii6h%2BPP9vPvlWIaYRSX6XqVFKuRwIwET9dwkybcKWIFU91E6vwTS76GLA7jm5TF%2FLbFExZ12u0XEkx8pFILd6q8%2BvZONW7A1hIhMnsnQihAnGyKsqR%2FJ6Bw2LckunJ9u31ZHp86FZNtimhokx05nw8%2FWbGD%2F1hKj75vcDnG7z6f6WRmlMyOwb2xhXAfxxdH8elF5uHmJOVQCKmGeW4B0ceolKnHFoTzN%2B9sBYQOzt

 pip install snowflake-connector-python[secure-local-storage]


9.40.7


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error

In [ ]:
# getting exposure data

event_df = snowflake_client.fetch(f"""
SELECT TO_TIMESTAMP(HOUR_TS) as HOUR_TS,
       CCY,
       ACCOUNT_TYPE,
       EVENT_TYPE,
       EXPOSURE_DIRECTION,
       SUM_EXPOSURE_AMOUNT
FROM ANALYTICS_DB.RPT_TREASURY.WEEKEND_FEATURES_EXPOSURE_EVENTS_HOURLY
WHERE HOUR_TS >= '2026-01-11 00:00:00'
    AND HOUR_TS <= '2026-01-13 00:00:00'
    order by hour_ts
    limit 500;
""")

event_df.head()

,hour_ts,ccy,account_type,event_type,exposure_direction,sum_exposure_amount
0,2026-01-12 22:00:00,TZS,BORDERLESS,BalanceReleaseReservedEvent,positive,4000.00
1,2026-01-12 22:00:00,BAM,BORDERLESS,BalanceReserveConversionEvent,negative,-209.63
2,2026-01-12 22:00:00,KWD,BORDERLESS,PlasticTransactionCreatedEvent,positive,5.74
3,2026-01-12 22:00:00,MXN,CUSTOMER,FixedRatePromisedEvent,negative,-2475599.12
4,2026-01-12 22:00:00,NZD,BORDERLESS,PlasticTransactionCreatedEvent,negative,-169.00


In [15]:
sorted(set(event_df.event_type))

['AcquiringPaymentConfirmedEvent',
 'AcquiringPaymentPnlEvent',
 'BalanceConversionPnlEvent',
 'BalanceInstantConversionEvent',
 'BalanceReleaseReservedEvent',
 'BalanceReserveConversionEvent',
 'BalanceTransactionV2PnlEvent',
 'BankTransactionPnlEvent',
 'CustomerTransferPnlEvent',
 'FixedRateGuaranteedEvent',
 'FixedRatePromisedEvent',
 'FixedTransferMarkReceivedEvent',
 'FixedTransferPayoutAllocatedEvent',
 'FixedTransferPayoutRollbackAllocatedEvent',
 'FixedTransferRollbackReceivedEvent',
 'InternalTransferPaidOutEvent',
 'JournalCustomerTransferAsLossEvent',
 'MarketOrderExecutedEvent',
 'NonPaymentRefundLinkedEvent',
 'PartnerExchangePnlEvent',
 'PartnerExchangeRegisteredEvent',
 'PayoutRestartedEvent',
 'PayoutStoppedEvent',
 'PlasticPayoutsV2PnlEvent',
 'PlasticTransactionCreatedEvent',
 'ResetCustomerTransferBalanceEvent',
 'ResetSchemeSettlementBalanceEvent',
 'RollbackFixedRateGuaranteedEvent',
 'RollbackFixedRatePromisedEvent',
 'RollbackPlasticTransactionCreatedEvent',
 'T

In [16]:
# getting ccy data

df = snowflake_client.fetch(f"""
    SELECT *, ((BID_OUTRIGHT+OFFER_OUTRIGHT)/2) as mid
    FROM RPT_TREASURY.INT_ASSET_PRICING__FX_QUOTE_SNAPSHOT_HOURLY
    WHERE CCY_PAIR IN ('USDPHP', 'EURUSD', 'GBPUSD', 'USDJPY', 'AUDUSD', 'NZDUSD','USDCAD', 'USDCNH', 'USDSGD')
    AND TENOR_CODE = 'SP'
    AND INSTRUMENT_TYPE = 'FXSpot'
    AND PROVIDER = 'BLOOMBERG'
    AND TIMESTAMP BETWEEN '2026-01-11 00:00:00' AND '2026-01-13 00:00:00'
""")
ccy_df = df.pivot(index='timestamp', columns='ccy_pair', values='mid')
ccy_df.head()

ccy_pair,AUDUSD,EURUSD,GBPUSD,NZDUSD,USDCAD,USDCNH,USDJPY,USDPHP,USDSGD
timestamp,,,,,,,,,
2026-01-11 00:00:00,0.6687,1.1636,1.3404,0.57315,1.39135,6.97595,157.89,59.2565,1.28745
2026-01-11 01:00:00,0.6687,1.1636,1.3404,0.57315,1.39135,6.97595,157.89,59.2565,1.28745
2026-01-11 02:00:00,0.6687,1.1636,1.3404,0.57315,1.39135,6.97595,157.89,59.2565,1.28745
2026-01-11 03:00:00,0.6687,1.1636,1.3404,0.57315,1.39135,6.97595,157.89,59.2565,1.28745
2026-01-11 04:00:00,0.6687,1.1636,1.3404,0.57315,1.39135,6.97595,157.89,59.2565,1.28745
